In [ ]:
!nvidia-smi

In [ ]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

In [ ]:
# !python3 -m pip install packages/ipywidgets-7.6.5.tar.gz -f ./ --no-index --no-deps

In [ ]:
# install dependencies uploaded in packages folder
# !python3 -m pip install packages/ipywidgets-7.6.5.tar.gz
import subprocess
import glob
import os
from tqdm import tqdm
package_files = glob.glob(os.path.join('packages', '*.whl')) + glob.glob(os.path.join('packages', '*.tar.gz'))
# print('Installing:', package_files)
for f in tqdm(package_files, desc="Installing packages..."):
    subprocess.call(f"pip install {f} -f ./ --no-index --no-deps | tqdm", shell=True)

In [ ]:
# !pip list

In [ ]:
%cd mainfile

In [ ]:
%pwd

In [ ]:
!python dataprep.py --generate --split_ratio 0.2

# Training

In [ ]:
!export CUDA_VISIBLE_DEVICES=5

In [ ]:
import torch
x = torch.randn(100,128).to(device="cuda:5")
sim_mat = torch.matmul(x, x.transpose(0, 1))
x.get_device()

### Phase 1 classifier

In [ ]:
!python main.py --do_train --augment \
                --train_list dataset/train.txt \
                --test_list dataset/val.txt \
                --model ResNetSE34V2 \
                --max_epoch 200 \
                --batch_size 128 \
                --nDataLoaderThread 2 \
                --criterion amsoftmax \
                --n_mels 64 \
                --margin 0.2\
                --scale 30\
                --nPerSpeaker 1 \
                --early_stop \
                --es_patience 30 \
                --device "cuda:1" \
                --initial_model exp/ResNetSE34V2/model/best_state_d2010_8686_percent.model

### Phase 2 metric loss

In [ ]:
!python main.py --do_train --augment \
                --train_list dataset/train.txt \
                --test_list dataset/val.txt \
                --model ResNetSE34V2 \
                --max_epoch 500 \
                --batch_size 32 \
                --nDataLoaderThread 2 \
                --criterion amsoftmaxproto \
                --margin 0.1\
                --scale 30\
                --nPerSpeaker 2 \
                --early_stop \
                --es_patience 40 \
                --initial_model exp/ResNetSE34V2/model/best_state_d2010_8686_percent.model \
                --device "cuda:1" 
#                 --n_mels 80 

# Inference

In [ ]:
!python main.py --do_infer --prepare \
                --device "cuda:1" \
                --model ResNetSE34V2 \
                --eval_frames 100 \
                --test_list dataset/val.txt \
                --cohorts_path checkpoints/cohorts_resnet34V2.npy \
#                 --initial_model_infer checkpoint/final_500.model

In [ ]:
!python main.py --do_infer --eval \
                --device "cuda:1"\
                --model ResNetSE34V2 \
                --test_list dataset/val.txt \
                --eval_frames 100 \
                --cohorts_path checkpoints/cohorts_resnet34V2.npy \
                --initial_model_infer exp/ResNetSE34V2/model/best_state.model \
                --scoring_mode cosine

In [ ]:
!python main.py --do_infer --test \
                --device "cuda:1" \
                --model ResNetSE34V2 \
                --eval_frames 100 \
                --cohorts_path checkpoints/cohorts_resnet34V2.npy \
                --test_threshold 0.3609619736671448 \
                --initial_model_infer exp/ResNetSE34V2/model/best_state.model \
                --scoring_mode cosine

In [ ]:
import csv
#check results
def check_result(path):
    ref = "exp/data_truth.txt"
    com = path
    
    ref_data = {}
    com_data = {}
    

    with open(ref, newline='') as rf:
        spamreader = csv.reader(rf, delimiter=' ')
        for row in tqdm(spamreader):
            key = f"{row[1]}/{row[-1]}"
            ref_data[key] = row[0]
            
    with open(com, newline='') as rf:
        spamreader = csv.reader(rf, delimiter=',')
        next(spamreader, None)
        for row in tqdm(spamreader):
            key = f"{row[0]}/{row[1]}"
            com_data[key] = row[2]
            
    assert len(ref_data)==len(com_data) or len(ref_data)==(len(com_data) + 1)
    count = 0
    for k, v in com_data.items():
        if ref_data[k] == v:
            count += 1
    
    print("Precision:", count*100/len(ref_data))

check_result("exp/ResNetSE34V2/result/submission_list_test.csv")
    

## run deploy

In [ ]:
!python server_test.py

In [ ]:
len(glob.glob("dataset/wavs/*/*.wav")) *0.8

In [ ]:
with open('dataset/train.txt', 'r') as f:
    print(len(f.readlines()))